# Create and Invoke Agent via Boto3 SDK

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

This notebook demonstrates the usage of the `bedrock-agent` and `bedrock-agent-runtime` boto3 clients to:
- Create an agent
- Create an action group
- Associate the agent with the action group and prepare it for use
- Create an agent alias
- Invoke the agent

We'll be utilizing Bedrock's Claude v3 Sonnet through the Boto3 API.

**Note:** *This notebook is designed to be run both within and outside of an AWS environment.*

### Prerequisites

Ensure you have an AWS account with permissions to:
- Create and manage IAM roles and policies
- Create and invoke AWS Lambda functions
- Create, read from, and write to Amazon S3 buckets
- Access and manage Amazon Bedrock agents and models
- Create and manage Amazon Glue databases and crawlers
- Execute queries and manage Amazon Athena workspaces

### Context

The following sections guide you through creating and invoking a Bedrock agent using the Boto3 SDK.

### Use Case

The notebook sets up an agent capable of crafting SQL queries from natural language questions. It then retrieves responses from the database, providing accurate answers to user inquiries. The diagram below outlines the high-level architecture of this solution.

![sequence-flow-agent](images/text-to-sql-architecture-Athena.png)

The Agent is designed to:
- Retrieve database schemas
- Execute SQL queries


In [ ]:
upgrade_output = !pip install --upgrade pip
install_boto3_output = !pip install boto3
##Ensure your boto3 and botocore libraries are up to date
upgrade_output_botocore_boto3= !pip install --upgrade boto3 botocore 

In [1]:
from dependencies.config import *

[2024-09-09 22:21:32,295] p37156 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


TheHistoryOfBaseball


In [2]:
!python ./dependencies/build_infrastructure.py

TheHistoryOfBaseball


[2024-09-09 22:21:42,710] p23080 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
Traceback (most recent call last):
  File "c:\Users\Jan\repos\vct-hackathon\text2sql\dependencies\build_infrastructure.py", line 11, in <module>
    s3bucket = s3_client.create_bucket(
  File "c:\Users\Jan\repos\vct-hackathon\.venv\lib\site-packages\botocore\client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "c:\Users\Jan\repos\vct-hackathon\.venv\lib\site-packages\botocore\client.py", line 1023, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (IllegalLocationConstraintException) when calling the CreateBucket operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.


In [3]:
from dependencies.config import *

list_agent=bedrock_agent_client.list_agents()['agentSummaries']
#print(list_agent)
#print(agent_name)
# Search f
agent_id = next((agent['agentId'] for agent in list_agent if agent['agentName'] == agent_name), None)

print(agent_id)

response = bedrock_agent_client.list_agent_aliases(
    agentId=agent_id,
)
response['agentAliasSummaries']
agentAliasId=next((agent['agentAliasId'] for agent in response['agentAliasSummaries'] if agent['agentAliasName'] == agent_alias_name), None)
agent_alias_id=agentAliasId
print(agent_alias_id)

3UDJO2PYAK
LERUFYH7QM


In [4]:
# ### Invoke Agent
# Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.
query_to_agent = """What was John Denny's salary in 1986?"""

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False


# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText=query_to_agent,
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)
logger.info(pprint.pprint(agentResponse))

[2024-09-09 23:48:00,093] p37156 {2468800596.py:20} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 09 Sep 2024 20:48:17 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ccc8b82d-6eec-11ef-9338-165afc3e56e7',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'c94136c0-189c-44aa-89c0-74a2eeafd71e'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'c94136c0-189c-44aa-89c0-74a2eeafd71e',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x000001E2BAD28400>,
 'contentType': 'application/json',
 'sessionId': 'ccc8b82d-6eec-11ef-9338-165afc3e56e7'}


In [5]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-09-09 23:48:11,104] p37156 {<timed exec>:11} INFO - {
  "agentAliasId": "LERUFYH7QM",
  "agentId": "3UDJO2PYAK",
  "agentVersion": "1",
  "sessionId": "ccc8b82d-6eec-11ef-9338-165afc3e56e7",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\"        You are an expert database querying assistant that can create simple and complex SQL queries to get the answers to questions about baseball players that you are asked. You first need to get the schemas for the table in the database to then query the database tables using a sql statement then respond to the user with the answer to their question andthe sql statement used to answer the question. Use the getschema tool f

CPU times: total: 15.6 ms
Wall time: 3.4 s


In [6]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

John Denny's salary in 1986 was $1,083,333.


In [7]:
#Create function to invoke agent
def invoke_agent(query):
    ## create a random id for session initiator id
    session_id:str = str(uuid.uuid1())
    enable_trace:bool = False
    end_session:bool = False
 

    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    event_stream = agentResponse['completion']
    print("Fetching answer...")
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)



In [8]:
invoke_agent("What year was Nolan Ryan inducted into the Hall of Fame?")

Fetching answer...


[2024-09-09 23:48:53,285] p37156 {1394403490.py:24} INFO - Final answer ->
Nolan Ryan was inducted into the Baseball Hall of Fame in 1999.


In [13]:
invoke_agent("What year was Nolan Ryan inducted into the Hall of Fame?")


Fetching answer...


[2024-09-09 23:52:59,034] p37156 {1394403490.py:24} INFO - Final answer ->
Nolan Ryan was inducted into the Baseball Hall of Fame in 1999.


In [10]:
invoke_agent("In what year did Hank Aaron hit the most home runs?")


Fetching answer...


[2024-09-09 23:49:31,822] p37156 {1394403490.py:24} INFO - Final answer ->
Based on the database tables provided in the schema, I could not find any tables containing yearly batting statistics like home runs for individual players. The available tables have information on players, salaries, awards and Hall of Fame voting, but no detailed stats. Without access to Hank Aaron's home run totals by year, I cannot determine the specific year he hit the most home runs over his career. My apologies, but I do not have enough information in this database to answer the original question.


In [11]:
#This query requires a join of two tables to be able to answer the question
invoke_agent("What player has received the most All-Star Game selections?")

Fetching answer...


[2024-09-09 23:50:12,663] p37156 {1394403490.py:24} INFO - Final answer ->
Unfortunately, I could not find any data in the database tables provided about players' All-Star game selections. The player_award and player_award_vote tables track various awards and award voting, but do not seem to have information on All-Star game selections specifically. Without that data, I cannot determine which player has received the most All-Star game selections. My apologies, but I do not have enough information in this database to answer the original question.


In [12]:
#This query should say there is no data available for this year!!
invoke_agent("What was Babe Ruth's salary in 1930?")


Fetching answer...


[2024-09-09 23:51:51,318] p37156 {1394403490.py:24} INFO - Final answer ->
Unfortunately, I could not find Babe Ruth's salary for the year 1930 in the database tables provided. The "salary" table does not seem to have any salary records for Babe Ruth.


In [14]:
invoke_agent("Who is the richest player in baseball history? ")


Fetching answer...


[2024-09-09 23:53:15,643] p37156 {1394403490.py:24} INFO - Final answer ->
Based on the salary data in the database, Alex Rodriguez appears to be the richest player in baseball history, having earned a salary of $33,000,000 in one season according to the query results.


In [15]:
invoke_agent("What was John Denny's salary in 1986? ")


Fetching answer...


[2024-09-09 23:53:38,447] p37156 {1394403490.py:24} INFO - Final answer ->
John Denny's salary in 1986 was $1,083,333.


In [ ]:
# This will stop the notebook's execution and wait for the user to press Enter
input("Press Enter to continue...")


## Clean up (optional)

The next steps are optional and delete the infrustcture we built. 



In [ ]:
!python ./dependencies/clean.py


## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You